In [42]:
import sys

sys.path.append('..')

In [10]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

# no line wrap
pd.set_option('display.expand_frame_repr', False)

data = 'generated'
channels = '*'
limit = 10000
batch_size = 64
lrs = [1e-3, 5e-4, 1e-4]
weight_decays = [0, 1e-4]
epochs = 20

# Define the directory and file pattern
# name = f'{model_name}_{args.data}_{args.channels}_{args.limit}_{args.num_moves}_{args.engine_prob}_{args.batch_size}_{args.lr}_{args.weight_decay}_{args.epochs}.csv'
# example = ConvLSTM_synthetic_12_10000_60_0.5_64_0.001_0.0001_20.csv
# pattern = os.path.join('../results', '*_processed_6_10000_40_*_64_*_*.csv')
pattern = os.path.join('../results', f'*_{data}_{channels}_{limit}_*_{batch_size}_*_{epochs}.csv')
files = glob.glob(pattern)

results = {}


for file in files:
  data = file.split('/')[-1].split('_')
  model = data[0]
  lr = float(data[-3])
  weight_decay = float(data[-2])
  channels = int(data[2])
  df = pd.read_csv(file)
  df = df[df['type'] == 'test']
  row = df[df['accuracy'] == df['accuracy'].max()].iloc[0].to_dict()
  del row['epoch']
  del row['loss']
  del row['type']
  row['lr'] = lr
  row['wd'] = weight_decay
  row['channels'] = int(channels)
  if model not in results:
    results[model] = row
  else:
    if row['accuracy'] > results[model]['accuracy']:
      results[model] = row

df = pd.DataFrame(results).T
df = df.sort_values('accuracy', ascending=False)
print(df)

                           accuracy  precision   recall        f1       auc       lr      wd  channels
Transformer(evals, times)   0.65000   0.654057  0.65000  0.649990  0.857609  0.00005  0.0001      12.0
Transformer(evals)          0.62075   0.640407  0.62075  0.613799  0.841409  0.00010  0.0001      12.0
ConvLSTM(evals, times)      0.61650   0.621578  0.61650  0.613030  0.845956  0.00010  0.0000      12.0
ConvLSTM(evals)             0.59500   0.605814  0.59500  0.592280  0.831604  0.00010  0.0001      12.0
Transformer(times)          0.58725   0.602299  0.58725  0.586330  0.824566  0.00010  0.0001      12.0
ConvLSTM(times)             0.56750   0.601834  0.56750  0.567901  0.820302  0.00100  0.0001      12.0
ConvLSTM                    0.53100   0.533511  0.53100  0.527696  0.771482  0.00100  0.0001      12.0
Transformer                 0.52025   0.537052  0.52025  0.513407  0.768331  0.00010  0.0001      12.0
